# Random Forest

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import *
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
spark = SparkSession.builder.appName("asheesh").getOrCreate()

22/04/05 09:11:55 WARN Utils: Your hostname, a-Lenovo-Legion-Y530-15ICH resolves to a loopback address: 127.0.1.1; using 192.168.1.4 instead (on interface wlp7s0)
22/04/05 09:11:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/04/05 09:11:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/05 09:11:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/04/05 09:11:56 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/04/05 09:11:56 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
# Read Ads Data
df = spark.read.csv("ads.csv", header="true", inferSchema="true")
cols=df.columns
cols.remove("_c0")
new_cols=cols.copy()
new_cols.remove("event")
df=df.select(cols)
df.show()

[Stage 1:========>                                                  (1 + 6) / 7]

+-------+---+---------+-----+
|genders|ads|age_group|event|
+-------+---+---------+-----+
|   MALE| a7|     1-15| IMPR|
| FEMALE| a1|     1-15|CLICK|
|   MALE| a4|    31-45| IMPR|
|   MALE| a4|    46-60| IMPR|
| FEMALE| a5|     1-15|CLICK|
|   MALE| a9|    46-60| IMPR|
| FEMALE| a8|    46-60| IMPR|
| FEMALE| a7|    46-60|CLICK|
|   MALE| a5|    31-45|CLICK|
|   MALE| a9|    31-45|CLICK|
|   MALE| a8|     1-15| IMPR|
| FEMALE| a9|    46-60| IMPR|
| FEMALE| a3|    46-60|CLICK|
| FEMALE| a4|    16-30| IMPR|
| FEMALE|a10|    16-30| IMPR|
| FEMALE| a2|    16-30|CLICK|
| FEMALE| a5|    16-30|CLICK|
| FEMALE| a1|    16-30| IMPR|
|   MALE| a1|    46-60| IMPR|
|   MALE| a7|     1-15|CLICK|
+-------+---+---------+-----+
only showing top 20 rows



In [3]:
# Split Data
splits = df.randomSplit([0.7, 0.3])
train_df,test_df = splits[0],splits[1]

In [5]:
# index the data
genders_indexer = StringIndexer(inputCol="genders", outputCol="gendersIndex")
ads_indexer = StringIndexer(inputCol="ads", outputCol="adsIndex")
age_group_indexer = StringIndexer(inputCol="age_group", outputCol="age_groupIndex")
event_indexer = StringIndexer(inputCol="event", outputCol="eventIndex")
vectorAssembler = VectorAssembler(inputCols=["gendersIndex","adsIndex","age_groupIndex"], outputCol="rawFeatures")


In [6]:
# Random Forest Classifier
rf = RandomForestClassifier(featuresCol = 'rawFeatures', labelCol = 'eventIndex')

In [7]:
# Define PipeLine
pipeline = Pipeline(stages=[genders_indexer,ads_indexer,age_group_indexer,event_indexer,vectorAssembler,rf])


In [8]:
# In-Momery Data Transformation and Random Forest Classifier Model Training
pipelineModel = pipeline.fit(train_df)

In [9]:
# Save pipeline
pipelineModel.write().overwrite().save("random_forest_classifier")
# dir(pipelineModel.write().overwrite())

22/04/05 09:12:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,004,614,439 bytes) of heap memory
Scaling row group sizes to 93.56% for 8 writers
22/04/05 09:12:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,004,614,439 bytes) of heap memory
Scaling row group sizes to 83.17% for 9 writers
22/04/05 09:12:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,004,614,439 bytes) of heap memory
Scaling row group sizes to 74.85% for 10 writers
22/04/05 09:12:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,004,614,439 bytes) of heap memory
Scaling row group sizes to 68.05% for 11 writers
22/04/05 09:12:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,004,614,439 bytes) of heap memory
Scaling row group sizes to 62.37% for 12 writers
22/04/05 09:12:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,004,614,439 bytes) of heap memory
Scaling row group sizes to 68.05% for 11 writers
22/04/05 09:12:49 WARN MemoryManager: Total allocation exceeds 95.

In [10]:
! ls -lrt random_forest_classifier

total 8
drwxrwxr-x 2 a a 4096 Apr  5 09:12 metadata
drwxrwxr-x 8 a a 4096 Apr  5 09:12 stages


In [71]:
# prediction
# new_cols.remove("event")

test_df=test_df.groupBy(new_cols).count()
predictions = pipelineModel.transform(test_df)
predictions.select(new_cols+["prediction"]).show(truncate=False)


+-------+---+---------+----------+
|genders|ads|age_group|prediction|
+-------+---+---------+----------+
|FEMALE |a3 |1-15     |0.0       |
|MALE   |a2 |1-15     |1.0       |
|FEMALE |a2 |46-60    |1.0       |
|FEMALE |a3 |16-30    |1.0       |
|MALE   |a5 |16-30    |1.0       |
|FEMALE |a10|16-30    |1.0       |
|FEMALE |a5 |31-45    |1.0       |
|MALE   |a2 |46-60    |1.0       |
|MALE   |a1 |46-60    |1.0       |
|MALE   |a4 |46-60    |1.0       |
|MALE   |a6 |1-15     |0.0       |
|MALE   |a1 |16-30    |1.0       |
|MALE   |a6 |16-30    |0.0       |
|MALE   |a6 |31-45    |0.0       |
|FEMALE |a8 |16-30    |1.0       |
|MALE   |a9 |16-30    |0.0       |
|MALE   |a8 |31-45    |1.0       |
|MALE   |a5 |31-45    |1.0       |
|MALE   |a2 |16-30    |1.0       |
|FEMALE |a1 |31-45    |0.0       |
+-------+---+---------+----------+
only showing top 20 rows



In [72]:
# from pyspark.ml.feature import IndexToString, StringIndexer
# converter = IndexToString(inputCol="prediction", outputCol="predicted_event")
# predictions_df=converter.transform(predictions)
# # predictions_df.select(new_cols+["predicted_event"]).show(truncate=False)
# predictions_df.show()